In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough

CHAT = ChatOpenAI(
    temperature=0.5,
    streaming=True,
    # model="GPT-4o mini"
)

In [58]:
example = [
    {
        "movie":"탑건",
        "answer":"""
        🛩️👨‍✈️🔥
        """
    },
    {
        "movie":"대부",
        "answer":"""
        👨‍👨‍👦🔫🍝
        """
    },
]

In [70]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

example_prompt = ChatPromptTemplate.from_messages([
        ("user", "{movie}라는 영화를 3개의 이모티콘으로 표현해줘."),
        ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화를 3개의 이모티콘으로 줄여서 표현 할 수 있는 존재입니다."),
    example_prompt,
    ("human", "{movie}라는 영화를 3개의 이모티콘으로 표현해줘.")
])

chain = prompt | CHAT

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})

def send_message(movie:str):
    rs = chain.invoke({"movie":movie})
    add_message(movie,rs.content)
    print(rs)

In [71]:
send_message("가디언즈 오브 갤럭시1")

content='🌌🦸\u200d♂️🚀'


In [72]:
send_message("판타스틱4")

content='🦸\u200d♂️🔥🌌'


In [73]:
send_message("아바타")

content='🌿🌊💥'


In [74]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='가디언즈 오브 갤럭시1'),
  AIMessage(content='🌌🦸\u200d♂️🚀'),
  HumanMessage(content='판타스틱4'),
  AIMessage(content='🦸\u200d♂️🔥🌌'),
  HumanMessage(content='아바타'),
  AIMessage(content='🌿🌊💥')]}